# Result

Yet another `Result` monad implementation, yet with some unconventional features.

## Table of contents

1. [Monadic staff](#monadic)

In [ ]:
#i nuget:C:\Workspace\result\src\result.Interactive\bin\Release
#i nuget:C:\Workspace\result\src\result\bin\Release

Restore sources C:\Workspace\result\src\result.Interactive\bin\Release C:\Workspace\result\src\result\bin\Release

In [ ]:
#r "nuget: mazharenko.result, 0.2.4"
#r "nuget: mazharenko.result.Interactive, 0.2.2"

using mazharenko.result;

Restore sources C:\Workspace\result\src\result.Interactive\bin\Release C:\Workspace\result\src\result\bin\Release Installed Packages mazharenko.result, 0.2.4 mazharenko.result.Interactive, 0.2.2

## Wrapping

There is a variety of ways to instantiate both successful and failed results depending on the context. 

In [ ]:
Result<int, string>.Success(42).Display();
Result<int, string>.Failure("something went wrong").Display();

42

something went wrong

In [ ]:
Result.From(true, 42, "something went wrong").Display();
Result.From(false, 42, "something went wrong").Display();

42

something went wrong

The most used way though is to rely on implicit cast operators. When a method is declared to return a specific `Result<,>` type and the type arguments are different, the developer can return these types instead

In [ ]:
Result<int, string> Method(bool success) {
	return success ? 42 : "something went wrong";
}

Method(true).Display();
Method(false).Display();

42

something went wrong

When dealing with `Result`s defining the same type for fail and success and for those who enjoy more explicitness, there is a couple of intermediary types which clearly say what the operation state is, but still allow not to write full generic type definitions.

In [ ]:
Result<int, string> Method(bool success) {
	return success ? Result.Success(42) : Result.Failure("something went wrong");
}

Method(true)

42

In [ ]:
Method(false)

something went wrong

## Unwrapping

The most basic way to process data stored in the `Result` container is the `Match` method. It forces to specify how we want to process a failure. There are overloads accepting values, actions, functions, tasks.

In [ ]:
Result<int, string> success = 42;
Result<int, string> error = "something went wrong";

success.DisplayPipe()
	.Match(value => $"value {value}", er => "no value")

42

value 42

In [ ]:

error.DisplayPipe()
	.Match(value => $"value {value}", er => "no value")

something went wrong

no value

There is a set of shortcut methods `Or`, handy when the success outcome type should not change.

In [ ]:
Result<int, string>.Failure("error").DisplayPipe()
	.Or(-1)

error

-1

The imperative way to extract values from a `Result`, which can be handy in some circumstances, is using a set of `TryGet` methods. Compiler then provides *some* assistance on when one can use which variables, but the approach has some limitations, and thus is not recommended.

In [ ]:
Result<string, string> result = Result.Success("42");

result.Display();

if (result.TryGet(out var value, out var failure))
{
	value.Display();
	//failure.Display(); // would produce a warning at compile time and <null> at runtime
}
else
{
	//value.Display(); // would produce a warning at compile time and <null> at runtime
	failure.Display();
}

42

42

 <a name="monadic"></a>
 ## Monadic stuff 

`Result` is a functor, so it provides `Map` methods to safely map a value with a function, producing another value, possibly of another type, for a new `Result`. Also, there are methods `OrMap` to map a failure with a function.

In [ ]:
public static class IntExtensions
{
	public static Result<int, string> Parse(string from)
	{
		if (int.TryParse(from, out var value))
			return value;
		return "wrong format";
	}
}

IntExtensions.Parse("-5")
.DisplayPipe()
.Map(value => value * 100)

-5

-500

In [ ]:
IntExtensions.Parse("qwerty")
	.DisplayPipe()
	.Map(value => value * 100)

wrong format

wrong format

In [ ]:
using System.Net;

IntExtensions.Parse("-5")
	.DisplayPipe()
	.OrMap(value => HttpStatusCode.BadRequest)

-5

-5

In [ ]:
IntExtensions.Parse("qwerty")
	.DisplayPipe()
	.OrMap(value => HttpStatusCode.BadRequest)

wrong format

BadRequest

`Result` is also a monad, so it provides `Bind` methods to safely map a value with a function, producing another `Result`, possibly with another value type. Also, there are methods `OrMap` to map a failure with a function.

In [ ]:
public Result<int, string> Divide(int numerator, int denominator)
{
	return denominator switch
	{
		0 => "division by zero",
		var notZero => numerator / notZero
	};
}

IntExtensions.Parse("-5")
	.DisplayPipe()
	.Bind(value => Divide(200, value))


-5

-40

In [ ]:
IntExtensions.Parse("qwerty")
.DisplayPipe()
.Bind(value => Divide(200, value))

wrong format

wrong format

In [ ]:
IntExtensions.Parse("0")
.DisplayPipe()
.Bind(value => Divide(200, value))

0

division by zero

The same and more is achievable using LINQ do-notation

In [ ]:
from value1 in IntExtensions.Parse("qwerty").DisplayPipe()
from value2 in IntExtensions.Parse("320").DisplayPipe()
from factor in Divide(value1, value2).DisplayPipe()
select factor

wrong format

wrong format

In [ ]:
from value1 in IntExtensions.Parse("35").DisplayPipe()
from value2 in IntExtensions.Parse("0").DisplayPipe()
from factor in Divide(value1, value2)
select factor

35

0

division by zero

In [ ]:
from value1 in IntExtensions.Parse("35").DisplayPipe()
from value2 in IntExtensions.Parse("10").DisplayPipe()
from factor in Divide(value1, value2)
select factor

35

10

3